In [1]:
!pip install timm

     |████████████████████████████████| 376 kB 808 kB/s 


In [2]:
import os
import sys
from glob import glob

import cv2
import warnings
import argparse
import random
import gc
import pandas as pd
from glob import glob
from tqdm import tqdm
import numpy as np
import torchvision
from torchvision import datasets, transforms
import torch
import torch.nn as nn
from torch.cuda.amp import autocast, GradScaler
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader, random_split, Dataset

import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

from sklearn.model_selection import KFold, GroupKFold, train_test_split, StratifiedKFold
warnings.simplefilter('ignore')

In [3]:
cat_path = glob("../input/cat-dataset/*/*.jpg")
dog_path = glob("../input/stanford-dogs-dataset/images/Images/*/*.jpg")
print(len(cat_path), len(dog_path))

9997 20580


In [4]:
dog_df = pd.DataFrame(columns=["file_path", "label"])
dog_df["file_path"] = dog_path
dog_df["label"] = 0

cat_df = pd.DataFrame(columns=["file_path", "label"])
cat_df["file_path"] = cat_path
cat_df["label"] = 1

df = pd.concat([dog_df, cat_df], axis=0)
df = df.reset_index(drop=True)

In [5]:
class CatDogDataset(Dataset):

    def __init__(self, df, transforms, type_):
        self.type = type_
        self.df = df
        self.transforms = transforms
        self.file_names = df['file_path'].values
        self.labels = df["label"].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        path = self.file_names[index]
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.transforms(image=image)['image'].float()
        label = torch.tensor(self.labels[index]).long()
        return image, label

In [6]:
image_size = 384
def train_transforms():
    train_transform = A.Compose(
        [
            A.RandomResizedCrop(int(image_size), int(
                image_size), scale=(0.90, 1.0)),
            A.RandomBrightnessContrast(p=0.2),
            A.ShiftScaleRotate(p=0.5),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(),
        ]
    )
    return train_transform

def valid_transforms():
    valid_transform = A.Compose(
        [
            A.Resize(image_size, image_size),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(),
        ]
    )
    return valid_transform

In [7]:
df["fold"] = 0
Fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=241)
for n, (train_index, val_index) in enumerate(Fold.split(df, df.label)):
    df.loc[val_index, 'fold'] = int(n)

In [8]:
df.head()

,file_path,label,fold
0,../input/stanford-dogs-dataset/images/Images/n...,0,3
1,../input/stanford-dogs-dataset/images/Images/n...,0,4
2,../input/stanford-dogs-dataset/images/Images/n...,0,3
3,../input/stanford-dogs-dataset/images/Images/n...,0,4
4,../input/stanford-dogs-dataset/images/Images/n...,0,0


In [9]:
class Custom2DCNN(nn.Module):
    def __init__(self):
        super(Custom2DCNN, self).__init__()
        model_name = "tf_efficientnet_b1"
        backborn = timm.create_model(model_name, pretrained=True, in_chans=3)
        if 'efficientnet' in model_name:
            n_features = backborn.classifier.in_features
            backborn.classifier = nn.Identity()
            self.backborn = backborn
        else:
            n_features = list(backborn.children())[-1].in_features
            backborn = list(backborn.children())[:-1]
            self.backborn = torch.nn.Sequential(*backborn)
        self.fc = nn.Linear(n_features, 1)

    def forward(self, x):
        x = self.backborn(x)
        x = self.fc(x)
        return x

In [10]:
def train_fn(epoch, model, loss_fn, optimizer, train_loader, scaler, device, scheduler=None):
    model.train()
    losses = AverageMeter()
    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for step, (imgs, image_labels) in pbar:
        imgs = imgs.to(device).float()
        image_labels = image_labels.to(device).float()

        with autocast():
            y_preds = model(imgs).squeeze(1)
            loss = loss_fn(y_preds, image_labels)
        scaler.scale(loss).backward()
        if ((step + 1) % 2 == 0) or ((step + 1) == len(train_loader)):
            # may unscale_ here if desired (e.g., to allow clipping unscaled gradients)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
        losses.update(loss.item(), 32)

    if ((step + 1) % 1 == 0) or ((step + 1) == len(train_loader)):
        description = f'epoch {epoch} loss: {losses.avg:.4f}'
        pbar.set_description(description)
    scheduler.step()

    return losses.avg

In [11]:
def valid_fn(epoch, model, loss_fn, val_loader, device, scheduler=None):
    model.eval()
    losses = AverageMeter()
    image_preds_all = []
    image_targets_all = []
    pbar = tqdm(enumerate(val_loader), total=len(val_loader))
    for step, (imgs, image_labels) in pbar:
        imgs = imgs.to(device).float()
        image_labels = image_labels.to(device).float()

        image_preds = model(imgs).squeeze(1)  # output = model(input)
        image_preds_all += [np.where(image_preds.detach().cpu().numpy() < 0.5, 0, 1)]
        image_targets_all += [image_labels.detach().cpu().numpy()]

        loss = loss_fn(image_preds, image_labels)
        losses.update(loss.item(), 32)


        if ((step + 1) % 1 == 0) or ((step + 1) == len(val_loader)):
            description = f'epoch {epoch} loss: {losses.avg:.4f}'
            pbar.set_description(description)

    image_preds_all = np.concatenate(image_preds_all)
    image_targets_all = np.concatenate(image_targets_all)
    print('validation class accuracy = {:.4f}'.format((image_preds_all == image_targets_all).mean()))

    return losses.avg, image_preds_all, image_targets_all

In [12]:
def get_score(y_true, y_pred):
    return accuracy_score(y_true, y_pred)

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [13]:
for fold in range(5):
    if fold >1:
        continue
    train = df[df.fold != fold]
    val = df[df.fold == fold]
    train_dataset = CatDogDataset(train, train_transforms(), 'train')
    val_dataset = CatDogDataset(val, valid_transforms(), 'train')
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=32,
        shuffle=True,
        num_workers=4,
        pin_memory=False,
        drop_last=True)
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=32,
        shuffle=False,
        num_workers=4,
        pin_memory=False,
        drop_last=True)
    
    loss = torch.nn.BCEWithLogitsLoss()
    model = Custom2DCNN().to(device)

    best_score = 0.
    
    scaler = GradScaler()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)    
    scheduler = CosineAnnealingLR(optimizer, T_max=20, eta_min=1e-5, last_epoch=-1)

    for epoch in range(20):
        train_loss = train_fn(
            epoch, model, loss, optimizer, train_loader, scaler, device, scheduler=scheduler)
        with torch.no_grad():
            valid_loss, valid_preds, valid_labels = valid_fn(
                epoch, model, loss, val_loader, device, scheduler=None)
        score = get_score(valid_labels, valid_preds)
        print(f'Epoch {epoch+1} - avg_train_loss: {train_loss:.4f}  avg_val_loss: {valid_loss:.4f}')
        print(f'Epoch {epoch+1} - Accuracy: {score}')
        if score > best_score:
            best_score = score
            print(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(), 'preds': valid_preds}, f'fold_{fold}_best.pth')
        torch.save(model.state_dict(), f'fold_{fold}_{epoch}.pth')


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_b1_aa-ea7a6ee0.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientnet_b1_aa-ea7a6ee0.pth
epoch 0 loss: 0.0039: 100%|██████████| 191/191 [01:04<00:00,  2.95it/s]


validation class accuracy = 0.9989
Epoch 1 - avg_train_loss: 0.0489  avg_val_loss: 0.0039
Epoch 1 - Accuracy: 0.9988547120418848
Epoch 1 - Save Best Score: 0.9989 Model


epoch 1 loss: 0.0025: 100%|██████████| 191/191 [01:01<00:00,  3.12it/s]


validation class accuracy = 0.9990
Epoch 2 - avg_train_loss: 0.0044  avg_val_loss: 0.0025
Epoch 2 - Accuracy: 0.9990183246073299
Epoch 2 - Save Best Score: 0.9990 Model


epoch 2 loss: 0.0038: 100%|██████████| 191/191 [01:03<00:00,  3.03it/s]

...

validation class accuracy = 0.9998
Epoch 18 - avg_train_loss: 0.0002  avg_val_loss: 0.0012
Epoch 18 - Accuracy: 0.9998363874345549



epoch 18 loss: 0.0011: 100%|██████████| 191/191 [01:12<00:00,  2.62it/s]


validation class accuracy = 0.9997
Epoch 19 - avg_train_loss: 0.0001  avg_val_loss: 0.0011
Epoch 19 - Accuracy: 0.99967277486911


epoch 19 loss: 0.0012: 100%|██████████| 191/191 [01:04<00:00,  2.97it/s]

validation class accuracy = 0.9998
Epoch 20 - avg_train_loss: 0.0001  avg_val_loss: 0.0012
Epoch 20 - Accuracy: 0.9998363874345549
